In [2]:
%reset

# Questions before start:
We have a bunch of csv files, where each one is divided into categories:
1. Country (belgium, england, france, germany, greece, italy, netherlands, portugal, scotland, spain, turkey)
2. League (0, 1, 2, 3)
3. Season (2000-2001, 2001-2002, ..., 2021-2022)

Should we merge all the csv files into one big dataframe? Or should we keep them separate? 
If we merge them, how do we keep track of the different leagues and seasons?  

## Max's thoughts:
We could merge them into one big dataframe, and provide analysis on the whole dataset. 
In case we need analyse a specific league or season, we can filter the dataframe accordingly.

**Before merging**, we should add a column for the league and season.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob


In [18]:
# csv_files = sorted(glob(f'data/train/**/*.csv', recursive=True))

def read_data(folder_path):
    df = pd.DataFrame()
    for file_path in sorted(glob(f'{folder_path}/**/*.csv', recursive=True)):
        csv_file = pd.read_csv(file_path)
        # TODO: turn into data format -> sort by date -> add season feature from csv name-> concat
        
        # Extract country, league, and season from file path
        parts = file_path.split('/')
        country = parts[-3]
        league = parts[-2]
        season = parts[-1].split('.')[0]

        # Add country, league, and season as features
        csv_file['Country'] = country
        csv_file['League'] = league
        csv_file['Season'] = season

        # Convert date to datetime format and sort by date
        csv_file['Date'] = pd.to_datetime(csv_file['Date'], format='%d/%m/%y', errors='coerce')
        csv_file = csv_file.sort_values(by='Date')
        
        df = pd.concat([df, csv_file], ignore_index=True, axis=0, join='outer', sort=False)
    return df

In [23]:
# Read all training data
train_df = read_data('data/train')

In [16]:
# # Specify the directory where your CSV files are located
# csv_files = sorted(glob('data/train/england/0/*.csv'))
# 
# # Create an empty dataframe to store the merged data
# merged_df = pd.DataFrame()
# 
# # Iterate through each CSV file
# for file in csv_files:
#     # Read each CSV file into a temporary dataframe
#     temp_df = pd.read_csv(file)
# 
#     # Merge the dataframes based on unique columns
#     test_df = pd.concat([test_df, temp_df], axis=0, join='outer', sort=False)


In [22]:
# Display unique column names and their count
unique_columns = train_df.columns.unique()
column_count = len(unique_columns)

print("Unique Column Names:")
print(unique_columns)

print("\nNumber of Unique Columns:", column_count)

Unique Column Names:
Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR',
       ...
       'SOD', 'SOA', 'LBAHH', 'LBAHA', 'LBAH', 'LB', 'LB.1', 'LB.2', 'HT',
       'AT'],
      dtype='object', length=171)

Number of Unique Columns: 171


In [24]:
# Drop empty columns starting with "Unnamed"
train_df = train_df.filter(regex='^(?!Unnamed).*$')


In [ ]:
# TODO: split columns into categories about match (results data), Match Statistics, and betting odds (as in notes.txt)